In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
%%javascript
// Fetch and decode the notebook filename
var notebookFilename = decodeURIComponent(
    window.document.body.dataset.notebookName
);
// Drop the filename into a Python variable
Jupyter.notebook.kernel.execute(
    `notebook_filename = "${notebookFilename}"`
);

In [ ]:
import os, sys
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(notebook_filename))), "lib/python"))

import numpy as np

from lib.cfg import Cfg
from lib.plot_builder import PlotBuilder
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

In [ ]:
##  configuration options
config_file = '../PicoPIC.json'

cfg = Cfg(config_file)

timestamp=1e-9
show_grid=False
use_cache=False
cmap='terrain'

cmp = 'position' # component
specie = 'electrons'
shape = 'rec'
# color limits (WARNING: clim_estimation may works incorrectly)
clim_estimation = 1 # cfg.get_clim_estimation()
clim = [-clim_estimation, clim_estimation]
image_interpolation = 'nearest'

autoselect = True

x_axis_label = r'$\mathit{Z (m)}$'
y_axis_label = r'$\mathit{R (m)}$'

plot_name = r'$\mathbf{Something\enspace to\enspace Debug}$'

In [ ]:
frame_number = 0
size = [0,0,0,0]
for i in cfg.probes:
    if i.component == cmp:
        frame_number = timestamp // (cfg.time[2] * i.schedule)
        size[0] = i.size[0]
        size[1] = i.size[1]
        size[2] = i.size[2]
        size[3] = i.size[3]

particles_file_template = os.path.join(os.path.dirname(config_file), "simulation_result", cmp, specie, shape, "{}-{}_{}-{}".format(size[0], size[2], size[1], size[3]), str(int(frame_number)))

value_r = particles_file_template + '_0.dat'
value_phi = particles_file_template + '_1.dat'
value_z = particles_file_template + '_2.dat'

data_r = np.fromfile(value_r, dtype=float, sep=' ')
data_phi = np.fromfile(value_phi, dtype=float, sep=' ')
data_z = np.fromfile(value_z, dtype=float, sep=' ')

# define plot builder
plot = PlotBuilder(0, 0,
                   fig_color=cfg.figure_color,
                   fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height,
                   fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=20,
                   x_ticklabel_end=1e-9, y_ticklabel_end=1e-9)

In [ ]:
# add subplots
plot_p = plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

# add timestamp to plot
plot.get_figure().suptitle("Time: {:.2e} s".format(timestamp), x=.85, y=.95)

plot_p.set_xlim([0,cfg.geometry_size[1]])
plot_p.set_ylim([0,cfg.geometry_size[0]])
# add images
plot_p.plot(data_z, data_r, 'b,')

plot.show()